In [1]:
path = "/Users/amrtamer/Documents/Internship/"
# path = "/content/drive/MyDrive/Internship/"

In [2]:
import sys
sys.path.append(path + 'Models')
sys.path.append(path + 'Models/Universal/uni2ts/myenv/lib/python3.10/site-packages/')
import utils as ut
import models

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import torch
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import os
import re
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import optuna
from xgboost import XGBRegressor
import joblib
from sklearn.multioutput import MultiOutputRegressor
import ast
import joblib
import matplotlib.pyplot as plt
import random
from matplotlib.backends.backend_pdf import PdfPages

In [4]:
data = pd.read_csv(path + r'data/Volve/Volve_cleaned_prepared.csv')
if "Unnamed: 0" in data.columns:
    data = data.drop(columns="Unnamed: 0")
data.head()

,well_name,WELL_BORE_CODE,date,prod_hrs,bhp,bht,dp_tubing,AVG_ANNULUS_PRESS,AVG_CHOKE_SIZE_P,whp,...,linear_regressor,EMA_short,EMA_medium,EMA_long,Rolling_short,Rolling_medium,Rolling_long,Lag_short,Lag_medium,Lag_long
0,7405,NO 15/9-F-1 C,2014-04-22,0.268405,1.029311,0.605973,0.396167,-2.016027,-0.533280,3.281836,...,-0.101892,-0.103579,-0.104768,-0.106929,-0.001355,-0.002618,-0.00519,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,7405,NO 15/9-F-1 C,2014-04-23,0.268405,0.845795,0.634777,0.239818,0.088113,-0.419656,2.849981,...,-0.101892,-0.103579,-0.104768,-0.106929,-0.001355,-0.002618,-0.00519,"[0.0, 0.0, -0.10230979661544379]","[0.0, 0.0, 0.0, 0.0, -0.10230979661544379]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,7405,NO 15/9-F-1 C,2014-04-24,0.268405,0.775025,0.639809,0.199890,-0.688434,-0.402874,2.607708,...,-0.101892,-0.009618,-0.053295,-0.079801,-0.001355,-0.002618,-0.00519,"[0.0, -0.10230979661544379, -0.10230979661544379]","[0.0, 0.0, 0.0, -0.10230979661544379, -0.10230...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.10..."
3,7405,NO 15/9-F-1 C,2014-04-25,0.268405,0.705033,0.642084,0.161493,1.145283,-0.379061,2.364038,...,-0.114759,-0.020781,-0.051611,-0.076565,-0.001355,-0.002618,-0.00519,"[-0.10230979661544379, -0.10230979661544379, 0...","[0.0, 0.0, -0.10230979661544379, -0.1023097966...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.1023097..."
4,7405,NO 15/9-F-1 C,2014-04-26,0.268405,0.625357,0.643889,0.117221,1.145283,-0.340115,2.088747,...,0.082297,-0.004428,-0.037199,-0.066767,-0.009496,-0.002618,-0.00519,"[-0.10230979661544379, 0.1777063666263608, -0....","[0.0, -0.10230979661544379, -0.102309796615443...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.102309796615..."


In [5]:
well_info = dict(set([(j, i) for i, j in data[['WELL_BORE_CODE', 'well_name']].values.tolist()])) # creating a dictionary of well identiification with {code: name} structure
well_info

{5769: 'NO 15/9-F-5 AH',
 7289: 'NO 15/9-F-15 D',
 7078: 'NO 15/9-F-11 H',
 5351: 'NO 15/9-F-14 H',
 5599: 'NO 15/9-F-12 H',
 7405: 'NO 15/9-F-1 C'}

In [6]:
identification_column = ['well_name', 'WELL_BORE_CODE', 'date']
single_feature_columns = ['prod_hrs', 'bhp', 'bht', 'dp_tubing', 'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht', 'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'linear_regressor', 'EMA_short', 'EMA_medium', 'EMA_long', 'Rolling_short', 'Rolling_medium', 'Rolling_long', 'oil_rate']
multiple_feature_columns = ['Lag_short', 'Lag_medium', 'Lag_long']
feature_columns = single_feature_columns + multiple_feature_columns
target_column = ['oil_rate']

In [7]:
data = data[identification_column + feature_columns]
data.shape

(7987, 26)

In [8]:
print(data[['prod_hrs', 'bhp', 'bht', 'dp_tubing', 'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht', 'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'linear_regressor', 'EMA_short', 'EMA_medium', 'EMA_long', 'Rolling_short', 'Rolling_medium', 'Rolling_long', 'oil_rate']].mean().round(4))
print(data[['prod_hrs', 'bhp', 'bht', 'dp_tubing', 'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht', 'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'linear_regressor', 'EMA_short', 'EMA_medium', 'EMA_long', 'Rolling_short', 'Rolling_medium', 'Rolling_long', 'oil_rate']].std().round(4))

prod_hrs                -0.0
bhp                     -0.0
bht                      0.0
dp_tubing               -0.0
AVG_ANNULUS_PRESS       -0.0
AVG_CHOKE_SIZE_P        -0.0
whp                     -0.0
wht                     -0.0
choke_size_percentage    0.0
oil_vol                  0.0
gas_vol                  0.0
water_vol                0.0
linear_regressor        -0.0
EMA_short                0.0
EMA_medium               0.0
EMA_long                 0.0
Rolling_short           -0.0
Rolling_medium           0.0
Rolling_long            -0.0
oil_rate                 0.0
dtype: float64
prod_hrs                 1.0
bhp                      1.0
bht                      1.0
dp_tubing                1.0
AVG_ANNULUS_PRESS        1.0
AVG_CHOKE_SIZE_P         1.0
whp                      1.0
wht                      1.0
choke_size_percentage    1.0
oil_vol                  1.0
gas_vol                  1.0
water_vol                1.0
linear_regressor         1.0
EMA_short                1.0

In [9]:
data

,well_name,WELL_BORE_CODE,date,prod_hrs,bhp,bht,dp_tubing,AVG_ANNULUS_PRESS,AVG_CHOKE_SIZE_P,whp,...,EMA_short,EMA_medium,EMA_long,Rolling_short,Rolling_medium,Rolling_long,oil_rate,Lag_short,Lag_medium,Lag_long
0,7405,NO 15/9-F-1 C,2014-04-22,0.268405,1.029311,0.605973,0.396167,-2.016027,-0.533280,3.281836,...,-0.103579,-0.104768,-0.106929,-0.001355,-0.002618,-0.005190,-0.102310,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,7405,NO 15/9-F-1 C,2014-04-23,0.268405,0.845795,0.634777,0.239818,0.088113,-0.419656,2.849981,...,-0.103579,-0.104768,-0.106929,-0.001355,-0.002618,-0.005190,-0.102310,"[0.0, 0.0, -0.10230979661544379]","[0.0, 0.0, 0.0, 0.0, -0.10230979661544379]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,7405,NO 15/9-F-1 C,2014-04-24,0.268405,0.775025,0.639809,0.199890,-0.688434,-0.402874,2.607708,...,-0.009618,-0.053295,-0.079801,-0.001355,-0.002618,-0.005190,0.177706,"[0.0, -0.10230979661544379, -0.10230979661544379]","[0.0, 0.0, 0.0, -0.10230979661544379, -0.10230...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.10..."
3,7405,NO 15/9-F-1 C,2014-04-25,0.268405,0.705033,0.642084,0.161493,1.145283,-0.379061,2.364038,...,-0.020781,-0.051611,-0.076565,-0.001355,-0.002618,-0.005190,-0.042238,"[-0.10230979661544379, -0.10230979661544379, 0...","[0.0, 0.0, -0.10230979661544379, -0.1023097966...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.1023097..."
4,7405,NO 15/9-F-1 C,2014-04-26,0.268405,0.625357,0.643889,0.117221,1.145283,-0.340115,2.088747,...,-0.004428,-0.037199,-0.066767,-0.009496,-0.002618,-0.005190,0.028674,"[-0.10230979661544379, 0.1777063666263608, -0....","[0.0, -0.10230979661544379, -0.102309796615443...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.102309796615..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7982,5769,NO 15/9-F-5 AH,2016-08-22,0.268405,-0.880933,-0.806036,-0.814175,1.149266,1.150153,-1.140330,...,-0.692802,-0.702316,-0.715245,-0.692202,-0.703021,-0.724022,-0.683109,"[-0.6852415797773639, -0.6943429268335604, -0....","[-0.6933422169244923, -0.687878486910018, -0.6...","[-0.7133929373653101, -0.7128670168291575, -0...."
7983,5769,NO 15/9-F-5 AH,2016-08-23,0.268405,-0.880933,-0.806036,-0.814175,1.158798,1.150153,-1.136215,...,-0.689514,-0.699464,-0.713067,-0.690182,-0.699482,-0.722275,-0.677835,"[-0.6943429268335604, -0.6840874763785847, -0....","[-0.687878486910018, -0.6852415797773639, -0.6...","[-0.7128670168291575, -0.704496114962062, -0.6..."
7984,5769,NO 15/9-F-5 AH,2016-08-24,0.268405,-0.880933,-0.806036,-0.814175,1.141085,1.150153,-1.137527,...,-0.687737,-0.697357,-0.711216,-0.688940,-0.696914,-0.720063,-0.679069,"[-0.6840874763785847, -0.6831086798251896, -0....","[-0.6852415797773639, -0.6943429268335604, -0....","[-0.704496114962062, -0.6974619277910209, -0.6..."
7985,5769,NO 15/9-F-5 AH,2016-08-25,0.268405,-1.334973,-1.285316,-1.327391,1.140689,1.150153,-1.140776,...,-0.686410,-0.695555,-0.709500,-0.685783,-0.695013,-0.718130,-0.678646,"[-0.6831086798251896, -0.6778348655598816, -0....","[-0.6943429268335604, -0.6840874763785847, -0....","[-0.6974619277910209, -0.6908660077334403, -0...."


In [10]:
for column in multiple_feature_columns:
    try:
        data[column] = data[column].apply(ast.literal_eval)
    except Exception:
        pass

In [11]:
data.isna().any()

well_name                False
WELL_BORE_CODE           False
date                     False
prod_hrs                 False
bhp                      False
bht                      False
dp_tubing                False
AVG_ANNULUS_PRESS        False
AVG_CHOKE_SIZE_P         False
whp                      False
wht                      False
choke_size_percentage    False
oil_vol                  False
gas_vol                  False
water_vol                False
linear_regressor         False
EMA_short                False
EMA_medium               False
EMA_long                 False
Rolling_short            False
Rolling_medium           False
Rolling_long             False
oil_rate                 False
Lag_short                False
Lag_medium               False
Lag_long                 False
dtype: bool

In [12]:
# Get the numerical index of the column 'oil_rate'
column_name = 'oil_rate'
column_index = data[feature_columns].columns.get_loc(column_name)
column_index

19

In [13]:
days_window = 5
predictions_days = 14
training_gap = predictions_days + days_window
batch_size = 96

In [ ]:
dataset = ut.produce_multivariate_dataset(data, days_window, predictions_days, single_feature_columns, multiple_feature_columns, target_column, well_info)

In [14]:
# dataset.to_pickle(path + "/data/Volve/all_data.pkl")
# dataset.to_csv(path + "/data/Volve/all_data.csv")
# dataset

In [15]:
train_size = 0.6
val_size = 0.2
test_size = 0.2

In [16]:
test_size_param1 = test_size
test_size_param2 = val_size / (1 - test_size_param1)

In [ ]:
data_train, data_val, data_test = ut.split_dataset(dataset, val_size, test_size, well_info, shuffle_train=True)

In [17]:
# data_train.to_pickle(path + "/data/Volve/data_train.pkl")
# data_val.to_pickle(path + "/data/Volve/data_val.pkl")
# data_test.to_pickle(path + "/data/Volve/data_test.pkl")

# data_train.to_csv(path + "/data/Volve/data_train.csv")
# data_val.to_csv(path + "/data/Volve/data_val.csv")
# data_test.to_csv(path + "/data/Volve/data_test.csv")

In [18]:
all_data = pd.read_pickle(path + "data/Volve/all_data.pkl")
data_train = pd.read_pickle(path + "data/Volve/data_train.pkl")
data_val = pd.read_pickle(path + "data/Volve/data_val.pkl")
data_test = pd.read_pickle(path + "data/Volve/data_test.pkl")

In [19]:
data_train.shape, data_val.shape, data_test.shape

((4730, 7), (1577, 7), (1573, 7))

In [20]:
check = True
for i, j in well_info.items():
    if len(all_data[all_data['well_name'] == i]) < 10:
        print("The input/output gap is too high for well", i)
        check = False
if check:
    print("The input/output gap is fine")

The input/output gap is fine


In [21]:
check=True
for i in [data_train, data_val, data_test]:
    for j in well_info.keys():
        if j not in set(list(i['well_name'])):
            print("Some datasets do not have specific well")
            check = False
            break
if check:
    print("All datasets have all wells")

All datasets have all wells


In [22]:
data_train['inputs'][0].shape, data_train['targets'][0].shape

((5, 38), (14, 1))

In [23]:
data_train

,well_name,inputs_dates,targets_dates,inputs_index,targets_index,inputs,targets
0,5351,"[2013-12-28, 2013-12-29, 2013-12-30, 2013-12-3...","[2014-01-02, 2014-01-03, 2014-01-04, 2014-01-0...","[6183, 6184, 6185, 6186, 6187]","[6188, 6189, 6190, 6191, 6192, 6193, 6194, 619...","[[0.2684054414666464, 0.6796976159384752, 0.46...","[[-0.4929884060060245], [-0.5018633150535997],..."
1,5599,"[2009-05-06, 2009-05-07, 2009-05-08, 2009-05-0...","[2009-05-11, 2009-05-12, 2009-05-13, 2009-05-1...","[1982, 1983, 1984, 1985, 1986]","[1987, 1988, 1989, 1990, 1991, 1992, 1993, 199...","[[-0.6648593334861422, 0.4864554330221693, 0.6...","[[2.5597611728030887], [2.5306894542768754], [..."
2,5351,"[2014-11-13, 2014-11-14, 2014-11-15, 2014-11-1...","[2014-11-18, 2014-11-19, 2014-11-20, 2014-11-2...","[6495, 6496, 6497, 6498, 6499]","[6500, 6501, 6502, 6503, 6504, 6505, 6506, 650...","[[0.2684054414666464, 0.738134882401413, 0.455...","[[-0.6891713748947265], [-0.695781903856089], ..."
3,7405,"[2014-07-15, 2014-07-16, 2014-07-17, 2014-07-1...","[2014-07-20, 2014-07-21, 2014-07-22, 2014-07-2...","[83, 84, 85, 86, 87]","[88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 9...","[[0.2684054414666464, 0.3177840898390375, 0.64...","[[-0.7268734393363852], [-0.5983770378894929],..."
4,7405,"[2015-06-16, 2015-06-17, 2015-06-18, 2015-06-1...","[2015-06-21, 2015-06-22, 2015-06-23, 2015-06-2...","[280, 281, 282, 283, 284]","[285, 286, 287, 288, 289, 290, 291, 292, 293, ...","[[0.2684054414666464, 0.6192371710627337, 0.64...","[[-0.8048592839445163], [-0.799994518985105], ..."
...,...,...,...,...,...,...,...
4725,7078,"[2016-01-15, 2016-01-16, 2016-01-17, 2016-01-1...","[2016-01-20, 2016-01-21, 2016-01-22, 2016-01-2...","[1305, 1306, 1307, 1308, 1309]","[1310, 1311, 1312, 1313, 1314, 1315, 1316, 131...","[[0.2684054414666464, 0.4323258240776714, 0.59...","[[-0.2096497082792693], [-0.2239800517639696],..."
4726,5599,"[2013-10-22, 2013-10-23, 2013-10-24, 2013-10-2...","[2013-10-27, 2013-10-28, 2013-10-29, 2013-10-3...","[3422, 3423, 3424, 3425, 3426]","[3427, 3428, 3429, 3430, 3431, 3432, 3433, 343...","[[0.2684054414666464, -1.739674518873222, -1.7...","[[-0.7503088106450555], [-0.7115700752297459],..."
4727,5351,"[2013-07-27, 2013-07-29, 2013-07-30, 2013-07-3...","[2013-08-04, 2013-08-05, 2013-08-06, 2013-08-0...","[6036, 6037, 6038, 6039, 6040]","[6041, 6042, 6043, 6044, 6045, 6046, 6047, 604...","[[0.2684054414666464, 0.6138642859264858, 0.47...","[[-0.4160433097861427], [-0.3920993164874174],..."
4728,5351,"[2009-11-04, 2009-11-05, 2009-11-06, 2009-11-0...","[2009-11-09, 2009-11-10, 2009-11-11, 2009-11-1...","[4829, 4830, 4831, 4832, 4833]","[4834, 4835, 4836, 4837, 4838, 4839, 4840, 484...","[[-0.4742447851268439, 0.6296190972456797, 0.5...","[[1.608625149967794], [1.5378065434017558], [1..."


In [24]:
train_dataset = ut.TimeSeriesDataset(data_train)
val_dataset = ut.TimeSeriesDataset(data_val)
test_dataset = ut.TimeSeriesDataset(data_test)

train_loader = DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)

In [25]:
visualization_test_dataset = ut.VisualizationTimeSeriesDataset(data_test)

visualization_test_loader = DataLoader(visualization_test_dataset, batch_size=int(len(test_dataset) / 50), shuffle=True)

In [27]:
def label_model_files(old_name="", new_name=""):
    if old_name != "":
        old_name = f'{old_name}_'
    if new_name != "":
        new_name = f'{new_name}_'
    config_path = path + r'Models/Global/XGBoost/Model/'
    if os.path.exists(config_path + f'{new_name}XGBoost.lib'):
        try:
            user_choice = int(input("File with new name already exists. Enter 1 if you want to overwrite the files. Otherwise press 2: "))
        except ValueError:
            print("Invalid input. Please enter 1 or 2.")
            return
        if user_choice == 1:
            try:
                os.rename(config_path + f'{old_name}XGBoost.lib', config_path + f'{new_name}XGBoost.lib')
                print("Files have been renamed successfully.")
            except FileNotFoundError:
                print("One or more files not found.")
        elif user_choice == 2:
            print("Operation cancelled by the user.")
        else:
            print("Invalid choice. Operation cancelled.")
    else:
        try:
            os.rename(config_path + f'{old_name}XGBoost.lib', config_path + f'{new_name}XGBoost.lib')
            print("Files have been renamed successfully.")
        except FileNotFoundError:
            print("One or more files not found.")

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    lr = trial.suggest_float('lr', 1e-4, 0.4, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 30)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 50)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
    model = models.XGBoost(n_estimators, lr, max_depth, min_child_weight, subsample, colsample_bytree)
    model.fit(torch.tensor(np.array(data_train['inputs'])), torch.tensor(np.array(data_train['targets'])).squeeze())
    outputs, val_loss = model.predict(torch.tensor(np.array(data_val['inputs'])))
    return val_loss

def hyperparameter_finetuning(objective, direction, n_trials):
    study = optuna.create_study(direction=direction)
    study.optimize(objective, n_trials=n_trials)
    best_params = study.best_params
    return best_params

In [28]:
%%capture
# Create a study and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)
best_params = study.best_params

best_params = hyperparameter_finetuning(objective, direction='minimize', n_trials=100)
n_estimators = best_params['n_estimators']
lr = best_params['lr']
max_depth = best_params['max_depth']
min_child_weight = best_params['min_child_weight']
subsample = best_params['subsample']
colsample_bytree = best_params['colsample_bytree']

[I 2024-07-23 20:31:27,034] A new study created in memory with name: no-name-436ff8a0-261d-4755-a055-145a143c224d
[W 2024-07-23 20:31:27,036] Trial 0 failed with parameters: {'n_estimators': 962, 'lr': 0.0993804160252913, 'max_depth': 14, 'min_child_weight': 28, 'subsample': 0.9670632173405105, 'colsample_bytree': 0.7113377942114552} because of the following error: NameError("name 'X_train' is not defined").
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.10/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/d8/xf5yjrpn5zvd1j56c42jys2w0000gn/T/ipykernel_10981/3517801789.py", line 64, in objective
    model.fit(X_train, Y_train)
NameError: name 'X_train' is not defined
[W 2024-07-23 20:31:27,039] Trial 0 failed with value None.


NameError: name 'X_train' is not defined

In [29]:
print('Best hyperparameters:', best_params)

NameError: name 'best_params' is not defined

In [26]:
n_estimators = 594
lr = 0.045570993763519016 
max_depth = 17
min_child_weight = 12
subsample = 0.7433625471337193 
colsample_bytree = 0.5992551725398402

In [27]:
model = models.XGBoost(n_estimators, lr, max_depth, min_child_weight, subsample, colsample_bytree)

In [28]:
model.fit(torch.tensor(np.array(data_train['inputs'])), torch.tensor(np.array(data_train['targets'])).squeeze())

/var/folders/d8/xf5yjrpn5zvd1j56c42jys2w0000gn/T/ipykernel_12412/715949354.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:277.)
  model.fit(torch.tensor(data_train['inputs']), torch.tensor(data_train['targets']).squeeze())


In [29]:
train_metrics = ut.evaluate(model, train_loader)
val_sizemetrics = ut.evaluate(model, val_loader)
test_metrics = ut.evaluate(model, test_loader)

Metrics on dataset:

Loss is 0.0001
MAE is 0.0059
RMSE is 0.0120
R² is 0.9999


Metrics on dataset:

Loss is 0.0261
MAE is 0.0805
RMSE is 0.1616
R² is 0.9757


Metrics on dataset:

Loss is 0.0056
MAE is 0.0568
RMSE is 0.0747
R² is 0.7667




In [34]:
model.save_model(path + r'Models/Global/XGBoost/Model/', "current")

TypeError: XGBoost.save_model() missing 1 required positional argument: 'directory'

In [31]:
label_model_files(old_name='current', new_name='best')

Invalid input. Please enter 1 or 2.


In [32]:
model = load_model(path + r'Models/Global/XGBoost/Model/', "best")

NameError: name 'load_model' is not defined

In [31]:
ut.produce_graphs(model, visualization_test_loader, data, data_test, days_window, predictions_days, path + "Models/Global/XGBoost/Graphs", well_info, test_size, column_index=column_index)
test_metrics = ut.evaluate(model, test_loader)

Metrics on dataset:

Loss is 0.0056
MAE is 0.0568
RMSE is 0.0747
R² is 0.7667


